In [19]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from faker import Faker
import requests
import os
import datetime as dt
import time
import faker


filePath = os.path.join(os.getcwd(), 'chromedriver-win64\chromedriver.exe')
url = 'https://www.taifex.com.tw/cht/3/pcRatio'
headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',                 
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36',                     
}


res = requests.get(url, headers=headers)

if res.status_code == 200:
    print('connect successfully')
    
    options = webdriver.ChromeOptions()    
    for key, value in headers.items():
        options.add_argument(key + '=' + value)

    # fake = Faker()
    # user_agent = fake.user_agent()
    # options.add_argument(f'user-agent={user_agent}')
    # options.add_argument("--headless") 
    driver = webdriver.Chrome(options=options)



    driver.get(url)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")  # 滾動網頁至底部 讓爬蟲程式不被發現
    driver.execute_script("window.scrollTo(0, 0)")  # 滾動網頁頂部 讓爬蟲程式不被發現


    startYear = 2023  # 設定爬取起始年份
    endYear = 2025 # 設定爬取結束年份

    df_dataframe = date_dataframe(startYear=startYear, endYear=endYear)

    dfc = pd.DataFrame()
    for i in range(0, len(df_dataframe)):
        
        
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")  # # 滾動網頁至底部 讓爬蟲程式不被發現
        driver.execute_script("window.scrollTo(0, 0)")  # 滾動網頁頂部 讓爬蟲程式不被發現


        startTagInput = driver.find_element(By.XPATH, '//*[@id="queryStartDate"]')
        startTagInput.clear()
        startTagInput.send_keys(df_dataframe.iloc[i, 0])


        endTagInput = driver.find_element(By.XPATH, '//*[@id="queryEndDate"]')
        endTagInput.clear()
        endTagInput.send_keys(df_dataframe.iloc[i, 1])

        btn = driver.find_element(By.XPATH, '//*[@id="button4"]')
        btn.click()



        table = driver.find_element(By.XPATH, '//*[@id="printhere"]/div[3]/div/table')
        col_names = [i.text for i in table.find_elements(By.TAG_NAME, 'th')]

        lst = []
        for i in table.find_elements(By.TAG_NAME, 'tr')[1:]:
            row_data = [j.text for j in i.find_elements(By.TAG_NAME, 'td')]
            lst.append([c for c in row_data])

        df = pd.DataFrame(lst, columns=col_names)
        dfc = pd.concat([dfc, df], axis=0)


    
        WebDriverWait(driver, 60).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="content"]/div[2]/div[3]/div/div[2]/a/img'))) 
        
        
    driver.close()

    dff = dfc.copy()
    dff[dff.columns[1:]] = dff[dff.columns[1:]].apply(lambda x: x.str.replace(',', ''), axis=1)
    dff[dff.columns[1:]] = dff[dff.columns[1:]].apply(pd.to_numeric, errors='coerce', axis=1)
    dff[dff.columns[0]] = pd.to_datetime(dff[dff.columns[0]])
    dff.sort_values(by=[dff.columns[0]], inplace=True)
    dff.reset_index(drop=True, inplace=True)
    
else:
    print('Website is not accessible.')

connect successfully


In [1]:
def date_dataframe(startYear, endYear):
    import calendar
    import datetime as dt
    import pandas as pd
    yrs = [i for i in range(startYear, endYear+1)]
    month = [i for i in range(1, 13)]

    lst_start = []
    lst_end = []

    for i in yrs:
        now = dt.datetime.now()
        yrs_now = now.year
        month_now = now.month
        if i != yrs_now:
            for j in month:
                start = dt.date(i, j, 1)
                end = dt.date(i, j, calendar.monthrange(i, j)[1])
                lst_start.append(start)
                lst_end.append(end)
        if i == yrs_now:
            for j in [k for k in range(1, month_now + 1)]:
                start = dt.date(i, j, 1)
                end = dt.date(i, j, calendar.monthrange(i, j)[1])
                lst_start.append(start)
                lst_end.append(end)

    df = pd.DataFrame()
    df['startDate'] = [i.strftime('%Y/%m/%d') for i in lst_start]
    df['endDate'] = [i.strftime('%Y/%m/%d') for i in lst_end]
    
    return df


In [23]:
print(dff.to_string())

            日期     賣權成交量      買權成交量  買賣權成交量比率%    賣權未平倉量    買權未平倉量  買賣權未平倉量比率%
0   2023-01-03  493351.0   496771.0      99.31  222208.0  207378.0      107.15
1   2023-01-04  563562.0   559566.0     100.71  135015.0  145210.0       92.98
2   2023-01-05  224444.0   228660.0      98.16  172712.0  184092.0       93.82
3   2023-01-06  285047.0   262552.0     108.57  214223.0  197219.0      108.62
4   2023-01-09  596922.0   484056.0     123.32  276672.0  193739.0      142.81
5   2023-01-10  479056.0   469606.0     102.01  301387.0  227327.0      132.58
6   2023-01-11  553048.0   555559.0      99.55  142750.0  131979.0      108.16
7   2023-01-12  147592.0   164753.0      89.58  161427.0  160363.0      100.66
8   2023-01-13  261440.0   305050.0      85.70  176152.0  175608.0      100.31
9   2023-01-16  195256.0   200493.0      97.39  189684.0  180133.0      105.30
10  2023-01-17  142037.0   147777.0      96.12  200645.0  191808.0      104.61
11  2023-01-30  349525.0   410532.0      85.14   894